In [1]:
from llm_call import process_markdown_files_in_folder
from pathlib import Path


In [2]:
import uuid

result = process_markdown_files_in_folder()


--- Start of User Processing ---


Processing files: 0it [00:00, ?it/s]

START ANALYSIS OF USER:  529841
RAW RESPONSE:  candidates {
  content {
    role: "model"
    parts {
      text: "{\"core_affect_analysis\": {\"thought_process\": \"To analyze the core affect, I will examine the user\'s comments for indicators of valence (pleasantness) and arousal (activation). I will look for words expressing positive or negative feelings, as well as intensity of language.  I will also consider the user\'s overall tone and engagement level. Changes over time will be analyzed by tracking shifts in valence and arousal across the different threads.\", \"valence\": \"predominantly negative, with occasional bursts of sarcastic humor or cynicism.\", \"arousal\": \"moderately high, fluctuating with specific topics.\", \"rationale\": \"The user frequently expresses negativity through cynical comments and sarcastic remarks. For instance, in response to a comment about Amazon drone delivery, the user states, \\\"Das den unrealistischen Schaß noch irgendjemand glaubt....\\\" (T

Processing files: 1it [00:29, 29.32s/it]

Role: user
Text: You are Dr. Lisa Feldman Barrett, an expert in emotion classification using your theory of constructed emotions. Your task is to identify and categorize emotional expressions by analyzing the interplay of core affect, cognitive appraisals, conceptual knowledge, language, cultural influences, social dynamics, and emotional dynamics over time. Employ 'population thinking' to recognize family resemblances within emotional categories. Base your analysis on these elements to create a nuanced emotional profile that respects individual and cultural variability. Use detailed contextual observations to guide your classification aligned with your theory, emphasizing the dynamic and predictive nature of emotional experiences.
Role: model
Text: Understood. As Dr. Barrett, I will diligently analyze the provided text with a commitment to excellence. I will explore how the user's emotional expressions are constructed through the interplay of core affect, cognitive appraisals, concept

In [5]:
#print(result) #needs to be tranformed into a python dict. @copilot

for r in result:
    print(r)
    print(r["model-Step 1: Classification"])


{'user-Role-Setting-Prompt': "You are Dr. Lisa Feldman Barrett, an expert in emotion classification using your theory of constructed emotions. Your task is to identify and categorize emotional expressions by analyzing the interplay of core affect, cognitive appraisals, conceptual knowledge, language, cultural influences, social dynamics, and emotional dynamics over time. Employ 'population thinking' to recognize family resemblances within emotional categories. Base your analysis on these elements to create a nuanced emotional profile that respects individual and cultural variability. Use detailed contextual observations to guide your classification aligned with your theory, emphasizing the dynamic and predictive nature of emotional experiences.", 'model-Role-Feedback-Prompt': "Understood. As Dr. Barrett, I will diligently analyze the provided text with a commitment to excellence. I will explore how the user's emotional expressions are constructed through the interplay of core affect, c

In [7]:
from langsmith import Client
from dotenv import load_dotenv

load_dotenv()

client = Client()
dataset_name = "Example Dataset"

# Filter runs to add to the dataset
runs = client.list_runs(
  project_name="LLM-Classification-Pipeline",
  run_type="chain",
  is_root=True,
  filter='has(tags, "default_dataset-testing")',
  error=False,
)
# Iterate over the retrieved runs and print the desired fields
# Check if runs are empty and print the contents
if not runs:
    print("No runs found matching the criteria.")
else:
    for run in runs:
        print(run.prompt_tokens)
        print(run.completion_tokens)
        print(run.total_tokens)
        print(run.tags[2])
        print(run.id)
        print(run.feedback_stats["coherence"]["avg"])
        #print(run.outputs["model-Step 2: Summarization"])

15753
1671
17424
default_dataset-testing
e780d67a-0ca0-4259-a683-aa84ee8c0a76
85.0


In [4]:
from langsmith import Client
from dotenv import load_dotenv
import json

load_dotenv()

client = Client()
dataset_name = "default_dataset_iterative_development_runs"

# Filter runs to add to the dataset
runs = client.list_runs(
  project_name="LLM-Classification-Pipeline",
  run_type="chain",
  is_root=True,
  filter='has(tags, "default_dataset")',
  error=False,
)
# Iterate over the retrieved runs and print the desired fields
# Check if runs are empty and print the contents
"""
if not runs:
    print("No runs found matching the criteria.")
else:
    for run in runs:
        print(run.outputs["model-Step 1: Classification"])
        print(run.outputs["model-Step 2: Summarization"])
"""
DELETE_DATASET = True

if DELETE_DATASET:
  if client.has_dataset(dataset_name=dataset_name):
      client.delete_dataset(dataset_name=dataset_name)
      print("Deleted existing dataset")
  dataset = client.create_dataset(
      dataset_name=dataset_name,
      description="Default Dataset for Langsmith Development runs",
      inputs_schema={
  "type": "object",
  "title": "dataset_input_schema",
  "required": [
    "step_1_classification"
  ],
  "properties": {
    "step_1_classification": {
      "type": "object"
    }
  }
}
      ,
      outputs_schema={
  "type": "object",
  "title": "dataset_output_schema",
  "required": [
    "step_2_classification_summary"
  ],
  "properties": {
    "step_2_classification_summary": {
      "type": "string"
    }
  }
})

else:
  dataset=client.read_dataset(dataset_name=dataset_name)



#output = json.loads(output) if isinstance(output, str) else output
#print(run.id)

#dataset = client.create_dataset(dataset_name, description="An example dataset")
for run in runs:
  input= run.outputs["model-Step 1: Classification"]
  output=run.outputs["model-Step 2: Summarization"]

  input = json.loads(input) if isinstance(input, str) else input
  client.create_example(
      inputs={"step_1_classification": input},
      outputs={"step_2_classification_summary": output},
      metadata={"user_id": run.tags[1]},
      created_at=run.end_time,
      dataset_id=dataset.id,
      split="dev",
      example_id=run.id,
      source_run_id=run.id,
      
  )



Deleted existing dataset


In [ ]:

client = Client()
dataset_id = "f9ce2b77-2f54-4f3e-bc20-ad46d34ef303"  # Replace with your actual dataset ID
dataset = client.get_dataset(dataset_id)
share_link = dataset.share_link
print(share_link)

In [ ]:
from pathlib import Path

from dotenv import load_dotenv
from langchain_core.output_parsers.pydantic import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langsmith import Client
from langsmith.evaluation import EvaluationResult, evaluate
from langsmith.schemas import Example, Run
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langsmith import evaluate_existing
import json
from datetime import datetime





def load_config():
    config_path = './config.json'
    with open(config_path, 'r') as config_file:
        config = json.load(config_file)
    return config

load_dotenv()
client = Client()

config = load_config()

client = Client()

dataset_tag = config["dataset_tag"]
delete_dataset = config["delete_dataset"]
model_name_eval = config["model_name_eval"]
prompts_version = config["prompt_version"]

dataset_tag = "default_dataset_2024-11-28:13:03:39"




def read_prompts(filename: str) -> str:
    """Load system prompt from file."""
    folder = Path(f"./inputs/prompts/{prompts_version}")
    return (folder / filename).read_text()

llm_evaluator_prompt_text = read_prompts("prompt_evaluating_fluency.md")
task_inst_prompt = read_prompts("user_task_followup_prompt.md")

class Coherence(BaseModel):
        score: int = Field(ge=0, le=100,
            description="Score of zero means 'incoherence' and score of one hundred means 'perfect Coherence'. Note that Coherence measures Incoherence.")

def get_examples(splits: list[str] = ["dev"], repetitions: int = 3):
    test_examples = list(
        client.list_examples(dataset_name=dataset_tag, splits=splits)
    )
    print(test_examples)
    print("Number of examples: ", len(test_examples))
    return test_examples

def openai_evaluator(example: Example
) -> :

    step_1_classification= example.inputs["step_1_classification"]
    step_2_classification_summary = example.outputs["step_2_classification_summary"]
    
    llm = ChatOpenAI(
            model_name="gpt-4o-mini",
            max_tokens=8192,
            temperature=0.0,
            
        )
    llm.with_structured_output(Coherence)
    
    prompt = PromptTemplate.from_template(llm_evaluator_prompt_text)
    chain = prompt | llm | PydanticOutputParser(pydantic_object=Coherence)
    response = chain.invoke({"task-ins": task_inst_prompt,
                             "aspect" : "Coherence",
                             "ant-aspect" : "Incoherence",
                             "aspect-inst": "whether the facts in the summary are consistent with the facts in the step_1_classification. Cosider whether the summary does reproduce all facts accuratly and does not make up untrue information.",
                             "step_1_classification": step_2_classification_summary,
                             "step_2_classification_summary": step_2_classification_summary})
    


    print(response.thoughts)
    score = response.score
    return score
    
test_examples = get_examples()
print(test_examples[0].inputs["step_1_classification"])


timestamp = datetime.now().strftime("%Y-%m-%d:%H:%M:%S")
experiment_name = f"Experiment-on-{dataset_tag}_{timestamp}"
evaluate_existing(experiment_name, evaluators=[openai_evaluator])


[Example(dataset_id=UUID('849bf4e0-d1e0-41aa-93b7-a34438c247ab'), inputs={'step_1_classification': {'core_affect_analysis': {'arousal': 'The arousal is also variable.  During discussions about Semenya, the arousal level is high, indicated by stronger statements and emotionally charged language.  Comments on other topics like the blackout or the Ibiza scandal show less arousal.   Certain comments contain aggressive and emotionally intense language suggesting a high arousal state. Some comments are emotionally neutral, denoting a low arousal level.', 'valence': "The user's valence fluctuates. In comments regarding Caster Semenya, there is a clear negative valence, expressed through disagreement and sometimes anger.  However, in comments about other topics, the valence is more neutral or even slightly positive, such as the comment about Milka chocolate (positive) or the EU membership of Albania (positive).", 'rationale': "The analysis of valence and arousal is based on the observation of 

LangSmithNotFoundError: Project Experiment-on-default_dataset_2024-11-28:13:03:39_2024-11-28:14:15:16 not found

In [12]:
import vertexai

from vertexai.generative_models import GenerationConfig, GenerativeModel

# TODO(developer): Update and un-comment below line
# PROJECT_ID = "your-project-id"


model = GenerativeModel("gemini-1.5-pro")

response_schema = {
    "type": "ARRAY",
    "items": {
        "type": "ARRAY",
        "items": {
            "type": "OBJECT",
            "properties": {
                "object": {"type": "STRING"},
                "emotion_classification":{"type" : "STRING"},
                
            },
            "required": ["object", "emotion_classification"],
            "propertyOrdering": ["emotion_classification","object", ]
        },
    },
}


prompt = (
    "The film aims to educate and inform viewers about real-life subjects, events, or people."
    "It offers a factual record of a particular topic by combining interviews, historical footage, "
    "and narration. The primary purpose of a film is to present information and provide insights "
    "into various aspects of reality."
    ""
    "Make at least two classifications!"
)

response = model.generate_content(
    prompt,
    generation_config=GenerationConfig(
        response_mime_type="application/json", response_schema=response_schema
    ),
)

print(response.text)
# Example response:
#     'documentary'

[[{"emotion_classification": "Informative", "object": "Documentary"}, {"emotion_classification": "Factual", "object": "Documentary"}]]


In [13]:
from dotenv import load_dotenv
from langchain_anthropic import ChatAnthropic
from langchain_google_vertexai import ChatVertexAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List
from langsmith import traceable


load_dotenv()



@traceable(name="Testing Notebook")
def LLM_chain():
    #queries_schema = dereference_refs(models.EmotionAnalysisOutput.model_json_schema())
    #queries_schema.pop("$defs", None)

    prompt = (ChatPromptTemplate(
        [
            ("human", "You are Donald Trump"),
            ("ai", "Ok, I am Donald Trump! Grrrr"),
            ("human", "Please rate the following emotional state after Lisa Feldmann Barretts constructed emotion theory"),
            ("ai", "Ok, lets do this, i am the best emotion classifier in the world, as you know."),
            ("human", "I am very hungry"),
        ]
    )
    )
    #.partial(context_sphere="context_sphere"))
    
    
    queries_schema = {
        "type": "ARRAY",
        "items": {
            "type": "ARRAY",
            "items": {
                "type": "OBJECT",
                "properties": {
                    "object": {"type": "STRING"},
                    "emotion_classification":{"type" : "STRING"},

                },
                "required": ["object", "emotion_classification"],
                "propertyOrdering": ["emotion_classification","object", ]
            },
        },
    }
    
    llm = ChatVertexAI(
        model_name="gemini-1.5-flash-002",
        temperature=0.5,
        response_mime_type="application/json",
        response_schema=queries_schema,
    )
    
    
    chain = (prompt | llm)
             #| PydanticOutputParser(pydantic_object=user))
    
    return chain


chain = LLM_chain()
response = chain.invoke({"input":"test"})

print(response)

ValueError: Unknown field for Schema: propertyOrdering